In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
# !pip3 install tensorflow==1.15.5

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

In [4]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [5]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model.univnet import discriminator
from malaya_speech.train.model.univnet import model
from malaya_boilerplate.train import config

In [6]:
hparams = config.HParams(**malaya_speech.config.univnet_config_c16)

In [7]:
discriminator = discriminator.Discriminator(hparams)

In [8]:
generator = model.Generator(hparams)

In [9]:
x = tf.random.normal(shape = (3, 16000, 1))

In [10]:
out = discriminator(x)
out

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


([([<tf.Tensor 'discriminator/multi_resolution_discriminator/discriminator_r/leaky_re_lu_49/LeakyRelu:0' shape=(3, 136, 513, 32) dtype=float32>,
    <tf.Tensor 'discriminator/multi_resolution_discriminator/discriminator_r/leaky_re_lu_50/LeakyRelu:0' shape=(3, 136, 257, 32) dtype=float32>,
    <tf.Tensor 'discriminator/multi_resolution_discriminator/discriminator_r/leaky_re_lu_51/LeakyRelu:0' shape=(3, 136, 129, 32) dtype=float32>,
    <tf.Tensor 'discriminator/multi_resolution_discriminator/discriminator_r/leaky_re_lu_52/LeakyRelu:0' shape=(3, 136, 65, 32) dtype=float32>,
    <tf.Tensor 'discriminator/multi_resolution_discriminator/discriminator_r/leaky_re_lu_53/LeakyRelu:0' shape=(3, 136, 65, 32) dtype=float32>,
    <tf.Tensor 'discriminator/multi_resolution_discriminator/discriminator_r/weight_normalization_5/conv2d_5/BiasAdd:0' shape=(3, 136, 65, 1) dtype=float32>],
   <tf.Tensor 'discriminator/multi_resolution_discriminator/discriminator_r/Reshape:0' shape=(3, 8840) dtype=float32>)

In [11]:
c = tf.placeholder(tf.float32, (None, None, 80))
z = tf.random.normal(shape = (tf.shape(c)[0], tf.shape(c)[1], 64))

In [13]:
z

<tf.Tensor 'random_normal_1:0' shape=(?, ?, 64) dtype=float32>

In [12]:
out_gen = generator(c, z = z)

In [14]:
out_gen

<tf.Tensor 'generator/sequential_1/activation/Tanh:0' shape=(?, ?, 1) dtype=float32>

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
out_ = sess.run(out)

In [19]:
%%time

out_gen_ = sess.run(out_gen, feed_dict = {c: np.random.normal(size = (2, 100, 80))})
out_gen_.shape

CPU times: user 922 ms, sys: 474 ms, total: 1.4 s
Wall time: 295 ms


In [25]:
%%time

out_gen_ = sess.run(out_gen, feed_dict = {c: np.random.normal(size = (2, 1000, 80))})
out_gen_.shape

CPU times: user 8.56 s, sys: 4.09 s, total: 12.6 s
Wall time: 2.63 s


(2, 256000, 1)

In [21]:
from malaya_speech.utils.featurization import universal_mel

In [26]:
universal_mel(out_gen_[0,:,0]).shape

(1001, 80)